In [15]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [16]:
columns = [
   "rating_good_or_bad","accept_rate_below_avg","accept_rate_above_avg","response_rate_below_avg","response_rate_above_avg","below_avg_num_of_reviews",
   "above_avg_num_of_reviews","below_avg_num_of_accom","above_avg_num_of_accom","below_avg_num_of_bedrooms","above_avg_num_of_bedrooms","below_avg_num_of_beds",
   "above_avg_num_of_beds","below_avg_num_of_baths","above_avg_num_of_baths","not_a_superhost","a_superhost","host_does_not_have_profile_pic",
   "host_does_have_profile_pic","host_does_not_have_identity_ver","host_does_have_identity_ver","not_instantly_bookable","instantly_bookable",
   "does_not_require_guest_prof_pic","requires_guest_profile_pic","does_not_require_guest_phone_verification","requires_guest_phone_verification",
   "rt_few_days","rt_within_day","rt_few_hours","rt_within_hours","location_ballard","location_beacon_hill","location_capital_hill",
   "location_cascade","location_central_area","location_delridge","location_downtown","location_interbay","location_lake_city","location_magnolia",
   "location_northgate","location_other","location_queen_anne","location_rainier_vally","location_seward_park","location_university_district",
   "location_west_seattle","pt_apartment","pt_b&b","pt_boat","pt_bungalow","pt_cabin","pt_rv","pt_chalet","pt_condo","pt_dorm","pt_house","pt_loft",
   "pt_other","pt_tent","pt_townhouse","pt_treehouse","pt_yurt","rt_entire_home","rt_private_room","rt_shared_room","bt_air_bed","bt_couch",
   "bt_futon","bt_pull_out_sofa","bt_real_bed","cp_flexible","cp_moderate","cp_strict"
]

# columns = [
# "id","host_id","booking_id","name","neighbourhood_group","city","state","zipcode","latitude","longitude","property_type","room_type",
# "accommodates","bathrooms","bedrooms","beds","bed_type","instant_bookable","cancellation_policy","require_guest_profile_picture","require_guest_phone_verification",
# "number_of_reviews","first_review","last_review","review_scores_rating","review_scores_accuracy","review_scores_cleanliness","review_scores_checkin","review_scores_communication",
# "review_scores_location","review_scores_value","reviews_per_month","host_name","host_since","host_listings_count","host_location","host_response_time","host_response_rate",
# "host_acceptance_rate","host_is_superhost","host_has_profile_pic","host_identity_verified","rating_good_or_bad","num_of_reivews_gt_avg","num_of_accom_gt_avg",
# "num_of_bedrooms_gt_avg","num_of_beds_gt_avg","num_of_baths_gt_avg","response_rate_gt_avg","accept_rate_gt_avg","accept_rate_below_avg","accept_rate_above_avg",
# "response_rate_below_avg","response_rate_above_avg","below_avg_num_of_reviews","above_avg_num_of_reviews","below_avg_num_of_accom","above_avg_num_of_accom",
# "below_avg_num_of_bedrooms","above_avg_num_of_bedrooms","below_avg_num_of_beds","above_avg_num_of_beds","below_avg_num_of_baths","above_avg_num_of_baths",
# "not_a_superhost","a_superhost","host_does_not_have_profile_pic","host_does_have_profile_pic","host_does_not_have_identity_ver","host_does_have_identity_ver",
# "not_instantly_bookable","instantly_bookable","does_not_require_guest_prof_pic","requires_guest_profile_pic","does_not_require_guest_phone_verification",
# "requires_guest_phone_verification","rt_few_days","rt_within_day","rt_few_hours","rt_within_hours","location_ballard","location_beacon_hill","location_capital_hill",
# "location_cascade","location_central_area","location_delridge","location_downtown","location_interbay","location_lake_city","location_magnolia","location_northgate",
# "location_other","location_queen_anne","location_rainier_vally","location_seward_park","location_university_district","location_west_seattle","pt_apartment",
# "pt_b&b","pt_boat","pt_bungalow","pt_cabin","pt_rv","pt_chalet","pt_condo","pt_dorm","pt_house","pt_loft","pt_other","pt_tent","pt_townhouse","pt_treehouse",
# "pt_yurt","rt_entire_home","rt_private_room","rt_shared_room","bt_air_bed","bt_couch","bt_futon","bt_pull_out_sofa","bt_real_bed","cp_flexible","cp_moderate","cp_strict"
# ]

target = ["rating_good_or_bad"]

In [58]:
# Load the data
file_path = Path('ml_final_data.csv')
df = pd.read_csv(file_path)
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()


# convert interest rate to numerical
# df['host_response_rate'] = df['host_response_rate'].str.replace('%', '')
# df['host_response_rate'] = df['host_response_rate'].astype('float') / 100
# df['host_acceptance_rate'] = df['host_acceptance_rate'].str.replace('%', '')
# df['host_acceptance_rate'] = df['host_acceptance_rate'].astype('float') / 100

df.reset_index(inplace=True, drop=True)

df.head()

,rating_good_or_bad,accept_rate_below_avg,accept_rate_above_avg,response_rate_below_avg,response_rate_above_avg,below_avg_num_of_reviews,above_avg_num_of_reviews,below_avg_num_of_accom,above_avg_num_of_accom,below_avg_num_of_bedrooms,...,rt_private_room,rt_shared_room,bt_air_bed,bt_couch,bt_futon,bt_pull_out_sofa,bt_real_bed,cp_flexible,cp_moderate,cp_strict
0,bad_review,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,good_review,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,good_review,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,bad_review,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,good_review,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [18]:
# Create our features
X = df.drop("rating_good_or_bad",axis=1)
X = pd.get_dummies(X)


# Create our target
y = df.loc[:,target].copy()

In [19]:
X.describe()

,accept_rate_below_avg,accept_rate_above_avg,response_rate_below_avg,response_rate_above_avg,below_avg_num_of_reviews,above_avg_num_of_reviews,below_avg_num_of_accom,above_avg_num_of_accom,below_avg_num_of_bedrooms,above_avg_num_of_bedrooms,...,rt_private_room,rt_shared_room,bt_air_bed,bt_couch,bt_futon,bt_pull_out_sofa,bt_real_bed,cp_flexible,cp_moderate,cp_strict
count,2651.000000,2651.000000,2651.000000,2651.000000,2651.000000,2651.000000,2651.000000,2651.000000,2651.000000,2651.000000,...,2651.000000,2651.000000,2651.000000,2651.000000,2651.000000,2651.000000,2651.000000,2651.000000,2651.000000,2651.000000
mean,0.007544,0.992456,0.007544,0.992456,0.708035,0.291965,0.595247,0.404753,0.743116,0.256884,...,0.297246,0.033572,0.005658,0.002641,0.017352,0.013203,0.961147,0.234628,0.356469,0.408902
std,0.086546,0.086546,0.086546,0.086546,0.454752,0.454752,0.490937,0.490937,0.436997,0.436997,...,0.457132,0.180159,0.075022,0.051328,0.130604,0.114163,0.193282,0.423846,0.479046,0.491724
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [20]:
# Check the balance of our target values
y["rating_good_or_bad"].value_counts()

good_review    1492
bad_review     1159
Name: rating_good_or_bad, dtype: int64

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)
print("Training:", Counter(y_train["rating_good_or_bad"]))
print("Test:", Counter(y_test["rating_good_or_bad"]))

Training: Counter({'good_review': 1121, 'bad_review': 867})
Test: Counter({'good_review': 371, 'bad_review': 292})


Oversampling

In [22]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler (random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
print(Counter(y_resampled["rating_good_or_bad"]))

Counter({'bad_review': 1121, 'good_review': 1121})


In [23]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [24]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)

from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test,y_pred)

0.6266107890558652

In [25]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm,index=["Actual Bad Review", "Actual Good Review"], columns=["Predicted Bad Review", "Predicted Good Review"])
cm_df

,Predicted Bad Review,Predicted Good Review
Actual Bad Review,192,100
Actual Good Review,150,221


In [26]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

 bad_review       0.56      0.66      0.60      0.61      0.63      0.39       292
good_review       0.69      0.60      0.66      0.64      0.63      0.39       371

avg / total       0.63      0.62      0.63      0.62      0.63      0.39       663



SMOTE Oversampling

In [27]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(X_train, y_train)
print(Counter(y_resampled["rating_good_or_bad"]))

Counter({'bad_review': 1121, 'good_review': 1121})


In [28]:
# Train the Logistic Regression model using the resampled data
modelS = LogisticRegression(solver='lbfgs', random_state=1)
modelS.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [29]:
# Calculated the balanced accuracy score
# Calculate predictions
y_pred = modelS.predict(X_test)

# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.6161937377690803

In [30]:
# Display the confusion matrix
cm2 = confusion_matrix(y_test, y_pred)
cm2_df =pd.DataFrame(cm2, index=["Actual Bad Review", "Actual Good Review"], columns=["Predicted Bad Review", "Predicted Good Review"])
cm2_df

,Predicted Bad Review,Predicted Good Review
Actual Bad Review,193,99
Actual Good Review,159,212


In [31]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

 bad_review       0.55      0.66      0.57      0.60      0.61      0.38       292
good_review       0.68      0.57      0.66      0.62      0.61      0.37       371

avg / total       0.62      0.61      0.62      0.61      0.61      0.38       663



Undersampling

In [32]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
print(Counter(y_resampled["rating_good_or_bad"]))

Counter({'bad_review': 867, 'good_review': 867})


In [33]:
# Train the Logistic Regression model using the resampled data
modelU = LogisticRegression(solver='lbfgs', random_state=1)
modelU.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [34]:
# Calculated the balanced accuracy score
y_pred = modelU.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6284338884170882

In [35]:
# Display the confusion matrix
cm3 = confusion_matrix(y_test, y_pred)
cm3_df =pd.DataFrame(cm3, index=["Actual Bad Review", "Actual Good Review"], columns=["Predicted Bad Review", "Predicted Good Review"])
cm3_df

,Predicted Bad Review,Predicted Good Review
Actual Bad Review,197,95
Actual Good Review,155,216


In [36]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

 bad_review       0.56      0.67      0.58      0.61      0.63      0.40       292
good_review       0.69      0.58      0.67      0.63      0.63      0.39       371

avg / total       0.64      0.62      0.63      0.62      0.63      0.39       663



Combo (over and under) sampling

In [37]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X,y)
print(Counter(y_resampled["rating_good_or_bad"]))

Counter({'bad_review': 600, 'good_review': 367})


In [38]:
# Train the Logistic Regression model using the resampled data
mlenn = LogisticRegression(solver='lbfgs',random_state=1)
mlenn.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [39]:
# Calculated the balanced accuracy score
y_pred = mlenn.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.616350662777388

In [40]:
# Display the confusion matrix
cm4 = confusion_matrix(y_test, y_pred)
cm4_df =pd.DataFrame(cm4, index=["Actual Bad Review", "Actual Good Review"], columns=["Predicted Bad Review", "Predicted Good Review"])
cm4_df

,Predicted Bad Review,Predicted Good Review
Actual Bad Review,223,69
Actual Good Review,197,174


In [41]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

 bad_review       0.53      0.76      0.47      0.63      0.60      0.37       292
good_review       0.72      0.47      0.76      0.57      0.60      0.35       371

avg / total       0.63      0.60      0.63      0.59      0.60      0.36       663



Random Forest

In [42]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
random_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
random_model.fit(X_train, y_train)
print(Counter(y_train["rating_good_or_bad"]))

Counter({'good_review': 1121, 'bad_review': 867})


In [43]:
# Calculated the balanced accuracy score
y_pred = random_model.predict(X_test)
from sklearn.metrics import confusion_matrix, accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.6040966658051177

In [44]:
# Display the confusion matrix
cm5 = confusion_matrix(y_test, y_pred)
cm5_df =pd.DataFrame(cm5, index=["Actual Bad Review", "Actual Good Review"], columns=["Predicted Bad Review", "Predicted Good Review"])
cm5_df

,Predicted Bad Review,Predicted Good Review
Actual Bad Review,182,110
Actual Good Review,154,217


In [45]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

 bad_review       0.54      0.62      0.58      0.58      0.60      0.37       292
good_review       0.66      0.58      0.62      0.62      0.60      0.36       371

avg / total       0.61      0.60      0.61      0.60      0.60      0.36       663



In [46]:
# List the features sorted in descending order by feature importance
importances = sorted(zip(random_model.feature_importances_,X.columns), reverse=True)
for importance in importances:
    print(f'{importance[1]}: {importance[0]*100:.1f}%:')

a_superhost: 4.0%:
not_a_superhost: 3.8%:
location_other: 3.7%:
rt_few_hours: 3.5%:
rt_within_hours: 3.3%:
cp_moderate: 3.0%:
location_capital_hill: 2.8%:
cp_strict: 2.8%:
rt_within_day: 2.7%:
above_avg_num_of_reviews: 2.6%:
pt_apartment: 2.6%:
location_central_area: 2.6%:
below_avg_num_of_reviews: 2.6%:
pt_house: 2.5%:
cp_flexible: 2.5%:
host_does_have_identity_ver: 2.5%:
not_instantly_bookable: 2.4%:
location_downtown: 2.4%:
host_does_not_have_identity_ver: 2.4%:
location_queen_anne: 2.4%:
above_avg_num_of_accom: 2.3%:
rt_entire_home: 2.2%:
below_avg_num_of_accom: 2.2%:
instantly_bookable: 2.2%:
rt_private_room: 2.2%:
location_ballard: 1.8%:
below_avg_num_of_beds: 1.7%:
above_avg_num_of_baths: 1.7%:
above_avg_num_of_beds: 1.7%:
below_avg_num_of_baths: 1.7%:
location_rainier_vally: 1.6%:
location_west_seattle: 1.5%:
below_avg_num_of_bedrooms: 1.5%:
above_avg_num_of_bedrooms: 1.4%:
location_university_district: 1.4%:
location_beacon_hill: 1.3%:
requires_guest_phone_verification: 1.0%:


Easy Ensemble 

In [47]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
easy_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)
easy_model.fit(X_train, y_train)
print(Counter(y_train["rating_good_or_bad"]))

Counter({'good_review': 1121, 'bad_review': 867})


In [48]:
# Calculated the balanced accuracy score
y_pred = easy_model.predict(X_test)
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test,y_pred)

0.6423863678322195

In [49]:
# Display the confusion matrix
cm6 = confusion_matrix(y_test, y_pred)
cm6_df =pd.DataFrame(cm6, index=["Actual Bad Review", "Actual Good Review"], columns=["Predicted Bad Review", "Predicted Good Review"])
cm6_df

,Predicted Bad Review,Predicted Good Review
Actual Bad Review,202,90
Actual Good Review,151,220


In [50]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

 bad_review       0.57      0.69      0.59      0.63      0.64      0.41       292
good_review       0.71      0.59      0.69      0.65      0.64      0.41       371

avg / total       0.65      0.64      0.65      0.64      0.64      0.41       663

